### IEX Code -------------------------------------------------------
1. Define the stock attributes
2. Define functions: filter attributes, fetch stock data, log events
3. Read company list from the redis datastore
4. Setup schedule
5. Fetch data at scheduled event.

In [1]:
#Imports
import redis
import urllib.request
import json
import numpy as np
import pandas as pd
import schedule
import time
import boto3
from datetime import date, datetime
import traceback
import os

kinesis = boto3.client('kinesis', region_name='us-east-1')

#Mongo
from pymongo import MongoClient


#Connect to Redis-DataStore
REDIS = redis.Redis(host='data_store')

In [2]:
#Setup Mongo and create the database and collection
client = MongoClient('db-data')
db = client['stock_tweets']
coll_reference = db.iex

In [3]:
########### Set up Feature Flag #####################

   
def get_feature_flag(feature):
    all_flags = pd.read_msgpack(REDIS.get("feature_flags"))
    
    try:
        return all_flags.get_value(feature, 'State')
        
    except:
        return 'Flag Not Found, not a valid feature flag'

In [8]:
############ Define Functions #################################
attributes = ['latestUpdate',
             'companyName',
             'latestPrice',
              'latestVolume',
              'marketCap',
              'open',
              'previousClose',
              'sector',
              'high',
              'low',
              'ytdChange',
              'peRatio',
              'week52High',
              'week52Low'
               ]

#Select interesting stock attributes.
def filter_stock_attributes(data):
    output = {}
    #Choose filter attributes
    for element in attributes:
        output[element]=data[element]
    
    #Convert time
    ctime = output['latestUpdate']/1000
    new_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(ctime))
    output['latestUpdate']= new_time
    
    return output


#Create function to fetch the stock data. This is to prepare for the schedule.

def fetch_stock_data(stocks=[]):
    try:
        #Only pull stock info if the DataFlag is set.
        if int(REDIS.get('Data_On')) == 1:

            for ticker in stocks:
                url = "https://api.iextrading.com/1.0/stock/{}/quote".format(ticker)
                response = urllib.request.urlopen(url)
                str_response = response.read().decode('utf-8')
                obj = json.loads(str_response)
                filtered = filter_stock_attributes(obj)
                filtered['Ticker'] = ticker
                
                #Add counter to count stocks. 
                REDIS.incr('IEX_Stock_Count')
                #<----- Insert to Kinesis Stream ------->
                if int(get_feature_flag('kinesis_stream_write'))==1:
                    response = kinesis.put_record(StreamName="IEX_Stream", Data=json.dumps(filtered), PartitionKey="partitionkey")
                
                #<------ Insert into MongoDb ----------->
                if int(get_feature_flag('database_stream_write'))==1:
                        coll_reference.insert_one(filtered)
                
                print('---------------------------------')
                #print(response)
                print(filtered)
    except:
        print(traceback.format_exc())
        #Send Error event
        send_event('IEX', 'Error', 'Error occured, check stdout')

#Setup Log

global past_stock_count
past_stock_count = 0




########### Redis Subscription setup ########################
queue = REDIS.pubsub()
#Subscribe to the queues one for the events and one for the log
queue.subscribe('event_queue')
queue.subscribe('log_queue')


#Serialize datetime.
def json_serial(obj):
    """JSON serializer for objects not serializable by default json code"""

    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError ("Type %s not serializable" % type(obj))
    


#Code to log to the event queue
def send_event(source, kind, message):
    event_time = datetime.now()
    event_time = json_serial(event_time)
    event = {
            "event_time": event_time,
            "source": source,
            "kind" : kind,
            "message" : message
            }
    payload = json.dumps(event)
    REDIS.publish('event_queue', payload)

def send_log(source, current_count, count_diff):
    log_time = datetime.now()
    log_time = json_serial(log_time)
    log = {
            "log_time": log_time,
            "source": source,
            "current_count" : current_count,
            "count_diff" : count_diff
            }
    payload = json.dumps(log)
    REDIS.publish('log_queue', payload)


#Send the log data to the Redis channel.
def log():
    #Need to log: Time, Source, Current Count, Count Diff
    #now = datetime.datetime.now()
    current_stock_count = int(REDIS.get('IEX_Stock_Count'))    
     
    global past_stock_count
    stock_count_diff = current_stock_count - past_stock_count
    past_stock_count = current_stock_count
    
    #Send the log event
    send_log(source='IEX',current_count = current_stock_count, count_diff=stock_count_diff)
    
    print('Logged Data')

In [ ]:
#Read Data-Store
companies = json.loads(REDIS.get('companies').decode())
stock_tickers = list(companies.keys())
REDIS.set('IEX_Stock_Count', 0)

#Setup Schedule
schedule.clear()
schedule.every(30).seconds.do(fetch_stock_data, stocks=stock_tickers)
schedule.every(35).seconds.do(log)

#Send Start event
send_event('IEX', 'Activity', 'Data Source Started')

#Execute
while True:
    schedule.run_pending()
    time.sleep(1)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


---------------------------------
{'latestUpdate': '2018-03-09 21:00:00', 'companyName': 'Apple Inc.', 'latestPrice': 179.98, 'latestVolume': 32123128, 'marketCap': 913220859740, 'open': 177.77, 'previousClose': 176.94, 'sector': 'Technology', 'high': 180, 'low': 177.39, 'ytdChange': 0.04481597585045861, 'peRatio': 19.56, 'week52High': 180.615, 'week52Low': 138.62, 'Ticker': 'AAPL', '_id': ObjectId('5aa57e97c8a4ce000c2464c4')}
---------------------------------
{'latestUpdate': '2018-03-09 21:00:00', 'companyName': 'Facebook Inc.', 'latestPrice': 185.23, 'latestVolume': 18290764, 'marketCap': 538093290404, 'open': 183.9, 'previousClose': 182.34, 'sector': 'Technology', 'high': 185.51, 'low': 183.21, 'ytdChange': 0.021000992172858574, 'peRatio': 30.12, 'week52High': 195.32, 'week52Low': 137.6, 'Ticker': 'FB', '_id': ObjectId('5aa57e98c8a4ce000c2464c5')}
---------------------------------
{'latestUpdate': '2018-03-09 21:00:00', 'companyName': 'Alphabet Inc.', 'latestPrice': 1160.04, 'lates

---------------------------------
{'latestUpdate': '2018-03-09 21:00:43', 'companyName': 'Citigroup Inc.', 'latestPrice': 76.11, 'latestVolume': 15931352, 'marketCap': 195607704080, 'open': 74.94, 'previousClose': 74.11, 'sector': 'Financial Services', 'high': 76.37, 'low': 74.6, 'ytdChange': 0.023534158149542767, 'peRatio': 14.28, 'week52High': 80.7, 'week52Low': 56.55, 'Ticker': 'C', '_id': ObjectId('5aa57e9cc8a4ce000c2464d7')}
---------------------------------
{'latestUpdate': '2018-03-09 21:05:15', 'companyName': 'United Technologies Corporation', 'latestPrice': 134.07, 'latestVolume': 4171922, 'marketCap': 107226276011, 'open': 132.19, 'previousClose': 131.26, 'sector': 'Industrials', 'high': 134.19, 'low': 131.51, 'ytdChange': 0.0461142322097378, 'peRatio': 20.13, 'week52High': 139.24, 'week52Low': 109.1, 'Ticker': 'UTX', '_id': ObjectId('5aa57e9cc8a4ce000c2464d8')}
---------------------------------
{'latestUpdate': '2018-03-09 21:00:00', 'companyName': 'Comcast Corporation Class

---------------------------------
{'latestUpdate': '2018-03-09 21:00:00', 'companyName': 'Tesla Inc.', 'latestPrice': 327.17, 'latestVolume': 5501309, 'marketCap': 55265537097, 'open': 324.49, 'previousClose': 329.1, 'sector': 'Consumer Cyclical', 'high': 328.49, 'low': 322.37, 'ytdChange': 0.020715689639035483, 'peRatio': -28.6, 'week52High': 389.61, 'week52Low': 242.781, 'Ticker': 'TSLA', '_id': ObjectId('5aa57ebec8a4ce000c2464ea')}
---------------------------------
{'latestUpdate': '2018-03-09 21:00:00', 'companyName': 'Cisco Systems Inc.', 'latestPrice': 45.37, 'latestVolume': 22204444, 'marketCap': 218570764892, 'open': 44.63, 'previousClose': 44.34, 'sector': 'Technology', 'high': 45.54, 'low': 44.61, 'ytdChange': 0.1675244467318579, 'peRatio': 20.91, 'week52High': 45.89, 'week52Low': 30.36, 'Ticker': 'CSCO', '_id': ObjectId('5aa57ebfc8a4ce000c2464eb')}
---------------------------------
{'latestUpdate': '2018-03-09 21:00:35', 'companyName': 'Exxon Mobil Corporation', 'latestPrice

---------------------------------
{'latestUpdate': '2018-03-09 21:00:00', 'companyName': 'Facebook Inc.', 'latestPrice': 185.23, 'latestVolume': 18290764, 'marketCap': 538093290404, 'open': 183.9, 'previousClose': 182.34, 'sector': 'Technology', 'high': 185.51, 'low': 183.21, 'ytdChange': 0.021000992172858574, 'peRatio': 30.12, 'week52High': 195.32, 'week52Low': 137.6, 'Ticker': 'FB', '_id': ObjectId('5aa57ee1c8a4ce000c2464fd')}
---------------------------------
{'latestUpdate': '2018-03-09 21:00:00', 'companyName': 'Alphabet Inc.', 'latestPrice': 1160.04, 'latestVolume': 2118910, 'marketCap': 806572946701, 'open': 1136, 'previousClose': 1126, 'sector': 'Technology', 'high': 1160.8, 'low': 1132.461, 'ytdChange': 0.08923943661971828, 'peRatio': 46.66, 'week52High': 1186.89, 'week52Low': 803.37, 'Ticker': 'GOOG', '_id': ObjectId('5aa57ee1c8a4ce000c2464fe')}
---------------------------------
{'latestUpdate': '2018-03-09 21:00:00', 'companyName': 'Alphabet Inc.', 'latestPrice': 1160.84, 'l

...

---------------------------------
{'latestUpdate': '2018-03-09 21:00:58', 'companyName': 'Wells Fargo & Company', 'latestPrice': 58.23, 'latestVolume': 18101652, 'marketCap': 284031609539, 'open': 57.23, 'previousClose': 56.72, 'sector': 'Financial Services', 'high': 58.27, 'low': 57, 'ytdChange': -0.04681617285971528, 'peRatio': 14.27, 'week52High': 66.31, 'week52Low': 49.27, 'Ticker': 'WFC', '_id': ObjectId('5aa57f9ac8a4ce000c246595')}
---------------------------------
{'latestUpdate': '2018-03-09 21:02:21', 'companyName': 'Visa Inc.', 'latestPrice': 124.51, 'latestVolume': 6396985, 'marketCap': 281007283136, 'open': 123.25, 'previousClose': 122.22, 'sector': 'Financial Services', 'high': 124.79, 'low': 122.87, 'ytdChange': 0.08732861758798358, 'peRatio': 35.78, 'week52High': 126.88, 'week52Low': 87.85, 'Ticker': 'V', '_id': ObjectId('5aa57f9ac8a4ce000c246596')}
---------------------------------
{'latestUpdate': '2018-03-09 21:01:07', 'companyName': 'Johnson & Johnson', 'latestPrice'

---------------------------------
{'latestUpdate': '2018-03-09 21:00:00', 'companyName': 'Amazon.com Inc.', 'latestPrice': 1578.89, 'latestVolume': 4508241, 'marketCap': 764351990167, 'open': 1564, 'previousClose': 1551.86, 'sector': 'Technology', 'high': 1578.94, 'low': 1559.08, 'ytdChange': 0.3279030453907033, 'peRatio': 346.25, 'week52High': 1578.94, 'week52Low': 833.5, 'Ticker': 'AMZN', '_id': ObjectId('5aa57fbcc8a4ce000c2465a8')}
---------------------------------
{'latestUpdate': '2018-03-09 21:00:00', 'companyName': 'Microsoft Corporation', 'latestPrice': 96.54, 'latestVolume': 36908605, 'marketCap': 743338001932, 'open': 95.33, 'previousClose': 94.43, 'sector': 'Technology', 'high': 96.54, 'low': 95, 'ytdChange': 0.12321116928446775, 'peRatio': 29.25, 'week52High': 96.54, 'week52Low': 64.12, 'Ticker': 'MSFT', '_id': ObjectId('5aa57fbcc8a4ce000c2465a9')}
---------------------------------
{'latestUpdate': '2018-03-09 21:00:23', 'companyName': 'Bank of America Corporation', 'latest

---------------------------------
{'latestUpdate': '2018-03-09 21:00:04', 'companyName': 'General Electric Company', 'latestPrice': 14.94, 'latestVolume': 64092422, 'marketCap': 129717685440, 'open': 14.62, 'previousClose': 14.52, 'sector': 'Industrials', 'high': 14.95, 'low': 14.51, 'ytdChange': -0.1690767519466074, 'peRatio': 14.23, 'week52High': 30.54, 'week52Low': 13.95, 'Ticker': 'GE', '_id': ObjectId('5aa57fc0c8a4ce000c2465bb')}
---------------------------------
{'latestUpdate': '2018-03-09 21:00:27', 'companyName': 'UnitedHealth Group Incorporated (DE)', 'latestPrice': 225.43, 'latestVolume': 4834581, 'marketCap': 218140251830, 'open': 226.11, 'previousClose': 224.73, 'sector': 'Healthcare', 'high': 226.52, 'low': 222.5, 'ytdChange': 0.019307288840658392, 'peRatio': 22.36, 'week52High': 250.79, 'week52Low': 162.74, 'Ticker': 'UNH', '_id': ObjectId('5aa57fc0c8a4ce000c2465bc')}
---------------------------------
{'latestUpdate': '2018-02-26 21:00:00', 'companyName': 'The Priceline 

Logged Data
---------------------------------
{'latestUpdate': '2018-03-09 21:00:00', 'companyName': 'Apple Inc.', 'latestPrice': 179.98, 'latestVolume': 32123128, 'marketCap': 913220859740, 'open': 177.77, 'previousClose': 176.94, 'sector': 'Technology', 'high': 180, 'low': 177.39, 'ytdChange': 0.04481597585045861, 'peRatio': 19.56, 'week52High': 180.615, 'week52Low': 138.62, 'Ticker': 'AAPL', '_id': ObjectId('5aa58001c8a4ce000c2465cd')}
---------------------------------
{'latestUpdate': '2018-03-09 21:00:00', 'companyName': 'Facebook Inc.', 'latestPrice': 185.23, 'latestVolume': 18290764, 'marketCap': 538093290404, 'open': 183.9, 'previousClose': 182.34, 'sector': 'Technology', 'high': 185.51, 'low': 183.21, 'ytdChange': 0.021000992172858574, 'peRatio': 30.12, 'week52High': 195.32, 'week52Low': 137.6, 'Ticker': 'FB', '_id': ObjectId('5aa58001c8a4ce000c2465ce')}
---------------------------------
{'latestUpdate': '2018-03-09 21:00:00', 'companyName': 'Alphabet Inc.', 'latestPrice': 116

---------------------------------
{'latestUpdate': '2018-03-09 21:00:43', 'companyName': 'Citigroup Inc.', 'latestPrice': 76.11, 'latestVolume': 15931352, 'marketCap': 195607704080, 'open': 74.94, 'previousClose': 74.11, 'sector': 'Financial Services', 'high': 76.37, 'low': 74.6, 'ytdChange': 0.023534158149542767, 'peRatio': 14.28, 'week52High': 80.7, 'week52Low': 56.55, 'Ticker': 'C', '_id': ObjectId('5aa58005c8a4ce000c2465e0')}
---------------------------------
{'latestUpdate': '2018-03-09 21:05:15', 'companyName': 'United Technologies Corporation', 'latestPrice': 134.07, 'latestVolume': 4171922, 'marketCap': 107226276011, 'open': 132.19, 'previousClose': 131.26, 'sector': 'Industrials', 'high': 134.19, 'low': 131.51, 'ytdChange': 0.0461142322097378, 'peRatio': 20.13, 'week52High': 139.24, 'week52Low': 109.1, 'Ticker': 'UTX', '_id': ObjectId('5aa58005c8a4ce000c2465e1')}
---------------------------------
{'latestUpdate': '2018-03-09 21:00:00', 'companyName': 'Comcast Corporation Class